In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"]="GEN-AIAPPWITHOPENAI"

In [3]:
#load the data of website
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    web_path = "https://docs.langchain.com/langsmith/generative-ui-react"
)

c:\Users\TANISH KALE\Desktop\langchain_embedding_openai_oilama etc\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
docs = loader.load()
print(docs)

[Document(metadata={'source': 'https://docs.langchain.com/langsmith/generative-ui-react', 'title': 'How to implement generative user interfaces with LangGraph - Docs by LangChain', 'language': 'en'}, page_content='How to implement generative user interfaces with LangGraph - Docs by LangChainSkip to main contentDocs by LangChain home pageLangSmithSearch...⌘KAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationTutorialsHow to implement generative user interfaces with LangGraphGet startedObservabilityEvaluationPrompt engineeringDeploymentPlatform setupReferenceOverviewTest locallyApp developmentCloud quickstartLocal development & testingConfigure app for deploymentApplication structureSetupDeployment componentsRebuild graph at runtimeInteract with a deployment using RemoteGraphAdd semantic search to your agent deploymentAdd TTLs to your applicationConfigure Agent Server for scaleImplement a CI/CD pipelineDeployment guidesCloudWith control planeStandalone serversTroubleshootingApp deve

In [5]:
#step 2 , create chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [6]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000, chunk_overlap = 300
)

chunks = splitter.split_documents(docs)
chunks


[Document(metadata={'source': 'https://docs.langchain.com/langsmith/generative-ui-react', 'title': 'How to implement generative user interfaces with LangGraph - Docs by LangChain', 'language': 'en'}, page_content='How to implement generative user interfaces with LangGraph - Docs by LangChainSkip to main contentDocs by LangChain home pageLangSmithSearch...⌘KAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationTutorialsHow to implement generative user interfaces with LangGraphGet startedObservabilityEvaluationPrompt engineeringDeploymentPlatform setupReferenceOverviewTest locallyApp developmentCloud quickstartLocal development & testingConfigure app for deploymentApplication structureSetupDeployment componentsRebuild graph at runtimeInteract with a deployment using RemoteGraphAdd semantic search to your agent deploymentAdd TTLs to your applicationConfigure Agent Server for scaleImplement a CI/CD pipelineDeployment guidesCloudWith control planeStandalone serversTroubleshootingApp deve

In [7]:
#step3: create an embedder

from langchain_openai import OpenAIEmbeddings
embeddder = OpenAIEmbeddings()

In [8]:
#step 4: get a vector db, embeded vec store in db at same time for time optimization
from langchain_community.vectorstores import FAISS
db = FAISS.from_documents(chunks,embeddder)


In [9]:
query = "What will LoadExternalComponent use by default"
res = db.similarity_search(query)
res[0].page_content

'Behind the scenes, LoadExternalComponent will fetch the JS and CSS for the UI components from LangSmith and render them in a shadow DOM, thus ensuring style isolation from the rest of your application.\n\u200bHow-to guides\n\u200bProvide custom components on the client side\nIf you already have the components loaded in your client application, you can provide a map of such components to be rendered directly without fetching the UI code from LangSmith.\nCopyconst clientComponents = {\n  weather: WeatherComponent,\n};\n\n<LoadExternalComponent\n  stream={thread}\n  message={ui}\n  components={clientComponents}\n/>;\n\n\u200bShow loading UI when components are loading\nYou can provide a fallback UI to be rendered when the components are loading.\nCopy<LoadExternalComponent\n  stream={thread}\n  message={ui}\n  fallback={<div>Loading...</div>}\n/>'

In [14]:
#Document Chain(new concept)

#install langchain_classic
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template(
    """
    Answer the question using only the context below:
    <context>
    {context}
    </context>
    
    """
)

llm = ChatOpenAI(model = "gpt-4o")
doc_chain = create_stuff_documents_chain(llm,prompt)
doc_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    Answer the question using only the context below:\n    <context>\n    {context}\n    </context>\n    \n    '), additional_kwargs={})])
| ChatOpenAI(profile={'max_input_tokens': 128000, 'max_output_tokens': 16384, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True}, client=<openai.resources.chat.completi

In [15]:
from langchain_core.documents import Document
doc_chain.invoke(
    {
        "input":"RemoveMessage you can remove an UI message from the state by calling",
        "context":[Document(page_content = "RemoveMessage you can remove an UI message from the state by calling")]
    }
)

"I'm sorry, but the context provided seems incomplete. Could you please provide more information or clarify your question?"

In [16]:
from langchain_classic.chains import create_retrieval_chain
retriever = db.as_retriever() # db - faiss


In [18]:
retrieval_chain = create_retrieval_chain(retriever,doc_chain)

In [22]:
retrieval_chain
# type(retrieval_chain) #a runable is a object which takes  input -> does work -> return output

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001DA73A938E0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    Answer the question using only the context below:\n    <context>\n    {context}\n    </context>\n    \n    '), additional_kwargs={})])
        

In [25]:
# get the response from the llm

response = retrieval_chain.invoke({
    "input": "emoveMessage you can remove an UI message from the state by calling"
})
response

{'input': 'emoveMessage you can remove an UI message from the state by calling',
 'context': [Document(id='df151a6f-f99d-4946-9400-d41efde27343', metadata={'source': 'https://docs.langchain.com/langsmith/generative-ui-react', 'title': 'How to implement generative user interfaces with LangGraph - Docs by LangChain', 'language': 'en'}, page_content='# push message\nmessage = push_ui_message("weather", {"city": "London"})\n\n# remove said message\ndelete_ui_message(message["id"])\nCopy// push message\nconst message = ui.push({ name: "weather", props: { city: "London" } });\n\n// remove said message\nui.delete(message.id);\n\n\u200bLearn more\n\nJS/TS SDK Reference\n\n\nEdit this page on GitHub or file an issue.\nConnect these docs to Claude, VSCode, and more via MCP for real-time answers.Was this page helpful?YesNoDeploy other frameworksPreviousLangSmith StudioNext⌘IDocs by LangChain home pagegithubxlinkedinyoutubeResourcesForumChangelogLangChain AcademyTrust CenterCompanyAboutCareersBlog

In [26]:
response['answer']

'How can you update a UI component message that has already been pushed?\n\nYou can update a UI component message that has already been pushed by calling `ui.push()` or `push_ui_message()` with the same ID as the UI message you wish to update.'